In [5]:
import pandas as pd
import pandas_ta as ta
import numpy as np

In [6]:
df = pd.read_csv('sp500.csv')
df = df.set_index('index')
df.index = pd.to_datetime(df.index)
df.head(5)

,Open,High,Low,Close,Adj Close,Volume,ticker
index,,,,,,,
2013-01-01,30.164520,30.722462,29.434908,30.436338,27.874155,24564398,A
2013-01-08,30.379112,31.559372,30.114449,31.144491,28.522694,21629576,A
2013-01-15,30.865522,31.781116,30.636623,31.580830,28.922304,17762849,A
2013-01-22,31.444921,32.582260,31.366238,32.317596,29.597048,18998819,A
2013-01-29,32.160229,32.660946,31.387697,31.659513,28.994360,21775387,A


In [7]:
from math import ceil

def week_of_month(dt):
    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

In [8]:
df['year'] = df.index.year
df['month'] = df.index.month
df['week_year'] = df.index.week
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['week_year'].astype(str) + '-1', format='%Y-%W-%w')
df['week_month'] = df['date'].apply(week_of_month)
df['week_month'] =df['week_month'] -1

/var/folders/5l/cbmvrvp95ldc5v4qdzr86h5m0000gn/T/ipykernel_23147/1504362151.py:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df['week_year'] = df.index.week


In [9]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,ticker,year,month,week_year,date,week_month
index,,,,,,,,,,,,
2013-01-01,30.164520,30.722462,29.434908,30.436338,27.874155,24564398,A,2013,1,1,2013-01-07,1
2013-01-08,30.379112,31.559372,30.114449,31.144491,28.522694,21629576,A,2013,1,2,2013-01-14,2
2013-01-15,30.865522,31.781116,30.636623,31.580830,28.922304,17762849,A,2013,1,3,2013-01-21,3
2013-01-22,31.444921,32.582260,31.366238,32.317596,29.597048,18998819,A,2013,1,4,2013-01-28,4
2013-01-29,32.160229,32.660946,31.387697,31.659513,28.994360,21775387,A,2013,1,5,2013-02-04,1


In [10]:
def add_obv(df):
    copy = df.copy()
    copy["OBV"] = (np.sign(copy["Close"].diff()) * copy["Volume"]).fillna(0).cumsum()
    return copy

In [11]:
def add_ema(df):
    copy = df.copy()
    copy['ema'] = copy['Close'].ewm(span=7, adjust=False).mean()
    return copy

In [24]:
def add_rsi(df, periods = 7, ema = True):
    copy = df.copy()
    copy['rsi'] = ta.rsi(df['Close'], length = 7)
    
    return copy

In [18]:
def add_mcda(df):
    copy = df.copy()
    k = copy['Close'].ewm(span=12, adjust=False, min_periods=12).mean()
    d = copy['Close'].ewm(span=26, adjust=False, min_periods=26).mean()

    macd = k - d
    macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
    macd_h = macd - macd_s
    
    copy['macd'] = macd
    copy['macd_h'] = macd_h
    copy['macd_s'] = macd_s
    
    return copy

In [25]:
df_features = add_obv(df)
df_features = add_ema(df_features)
df_features = add_rsi(df_features)
df_features = add_mcda(df_features)
df_features

,Open,High,Low,Close,Adj Close,Volume,ticker,year,month,week_year,date,week_month,OBV,ema,rsi,macd,macd_h,macd_s
index,,,,,,,,,,,,,,,,,,
2013-01-01,30.164520,30.722462,29.434908,30.436338,27.874155,24564398,A,2013,1,1,2013-01-07,1,0.000000e+00,30.436338,NaN,NaN,NaN,NaN
2013-01-08,30.379112,31.559372,30.114449,31.144491,28.522694,21629576,A,2013,1,2,2013-01-14,2,2.162958e+07,30.613377,NaN,NaN,NaN,NaN
2013-01-15,30.865522,31.781116,30.636623,31.580830,28.922304,17762849,A,2013,1,3,2013-01-21,3,3.939242e+07,30.855240,NaN,NaN,NaN,NaN
2013-01-22,31.444921,32.582260,31.366238,32.317596,29.597048,18998819,A,2013,1,4,2013-01-28,4,5.839124e+07,31.220829,NaN,NaN,NaN,NaN
2013-01-29,32.160229,32.660946,31.387697,31.659513,28.994360,21775387,A,2013,1,5,2013-02-04,1,3.661586e+07,31.330500,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-06,166.360001,166.369995,157.869995,158.330002,158.330002,9923300,ZTS,2023,2,6,2023-02-06,1,4.075275e+11,159.378219,52.358887,-0.954834,2.952760,-3.907594
2023-02-13,158.139999,176.660004,158.080002,172.029999,172.029999,12298000,ZTS,2023,2,7,2023-02-13,2,4.075398e+11,162.541164,66.820897,0.308841,3.373148,-3.064307
2023-02-20,171.529999,172.080002,163.910004,166.330002,166.330002,7507300,ZTS,2023,2,8,2023-02-20,3,4.075323e+11,163.488374,58.239373,0.840680,3.123989,-2.283310
